In [1]:
from fvgp import gp
import numpy as np
from numpy.typing import ArrayLike
import holoviews as hv
from ratingcurve.ratingmodel import PowerLawRating
import pymc as pm
import arviz as az
import pandas as pd
import hvplot.pandas

Equation for a broken power law with $n$ breaks is:

\begin{equation}
f(x) = \begin{cases}
            Ax^{\alpha_0} & {\rm if}\ x \leq x_1 \\
            Ax^{\alpha_1}x_1^{\alpha_0 - \alpha_1} & {\rm if}\ x_1 < x \leq x_2 \\
            ... \\
            Ax^{\alpha_n} {\displaystyle \prod^{n}_{i=1}} x_i^{\alpha_{i-1} - \alpha_{i}} & {\rm if}\ x > x_n
       \end{cases},
\end{equation}

where $A$ is the scaling amplitude, $\alpha_0$ is the power law slope before the first breakpoint ($x_1$), $\alpha_i$ are the power law slopes between breakpoints $x_{i}$ and $x_{i+1}$ with $i$ being the $i$th breakpoint and $i < n$, and $\alpha_n$ is the power law slope after the last breakpoint ($x_n$).

In log-space this translates to:

\begin{equation}
\log(f(x)) = \begin{cases}
            \log(A) + \alpha_0 \log(x) & {\rm if}\ x \leq x_1 \\
            \log(A) + \alpha_1 \log(x) + (\alpha_0 - \alpha_1) \log(x_1) & {\rm if}\ x_1 < x \leq x_2 \\
            ... \\
            \log(A) + \alpha_n \log(x) + {\displaystyle \sum^{n}_{i=1}} (\alpha_{i-1} - \alpha_{i}) \log(x_i) & {\rm if}\ x > x_n
       \end{cases}.
\end{equation}

In [2]:
def BrokenPowerLaw(x: ArrayLike, amplitude: float, breaks: ArrayLike, alphas: ArrayLike) -> np.ndarray:
    try:
        if len(breaks) != len(alphas) - 1:
            raise ValueError("Dimensional mismatch. There should be one more alpha than there are breaks.")
    except TypeError:
        raise TypeError("Breaks and alphas should be array-like.")
    if any(breaks < np.min(x)) or any(breaks > np.max(x)):
        raise ValueError("One or more break points fall outside given x bounds.")

    alphas_array = np.array(alphas)
    cumprod = np.cumprod(breaks ** (alphas_array[:-1] - alphas_array[1:]))

    fx = amplitude * np.array(x) ** alphas[0]
    for i in range(len(breaks)):
        if i < len(breaks) - 1:
            fx = np.where((x > breaks[i]) & (x <= breaks[i+1]), amplitude * x ** alphas[i+1] * cumprod[i], fx)
        else:
            fx = np.where(x > breaks[i], amplitude * x ** alphas[i+1] * cumprod[i], fx)
           
    return fx

Equation for a smoothly broken power law with $n$ breaks is:

\begin{equation}
    f(x) = Ax^{\alpha_0} {\displaystyle \prod^{n}_{i=1}} \Bigg(1 + \bigg(\frac{x}{x_i}\bigg)^{1/\Delta}\Bigg)^{(\alpha_{i+1} - \alpha_i)\Delta},
\end{equation}

where $A$ is the scaling amplitude, $\alpha_0$ is the power law slope before the first breakpoint ($x_1$), $\alpha_i$ are the power law slopes between breakpoints $x_{i}$ and $x_{i+1}$ with $i$ being the $i$th breakpoint and $i < n$, $\alpha_n$ is the power law slope after the last breakpoint ($x_n$), and $\Delta$ is the parameter controlling the smoothness of the break. Smaller (non-negative) values of $\Delta$ yield a sharper break, and larger values yield a smoother break.

This equation was adapted from Equation 1 in [Caballero et al. 2023](https://arxiv.org/pdf/2210.14891.pdf).

In log-space:

\begin{equation}
    \log(f(x)) = \log(A) + \alpha_0 \log(x) + {\displaystyle \sum^{n}_{i=1}} (\alpha_{i+1} - \alpha_i)\Delta \Bigg(1 + \bigg(\frac{x}{x_i}\bigg)^{1/\Delta}\Bigg),
\end{equation}

In [3]:
def SmoothlyBrokenPowerLaw(x: ArrayLike, amplitude: float, breaks: ArrayLike, alphas: ArrayLike, delta: float) -> np.ndarray:
    try:
        if len(breaks) != len(alphas) - 1:
            raise ValueError("Dimensional mismatch. There should be one more alpha than there are breaks.")
    except TypeError:
        raise TypeError("Breaks and alphas should be array-like.")
    if any(breaks < np.min(x)) or any(breaks > np.max(x)):
        raise ValueError("One or more break points fall outside given x bounds.")

    breakpoint_array = np.expand_dims(breaks, 0)
    alphas_array = np.array(alphas)
    alphas_diff_array = np.expand_dims(alphas_array[1:] - alphas_array[:-1], 0)
    x_array = np.expand_dims(x, 1)
    prod_array = (1 + (x_array/breakpoint_array) ** (1/delta)) ** (alphas_diff_array * delta)
    fx = amplitude * x ** alphas[0] * np.prod(prod_array, axis=1)
    return fx

Now that we have the power law functions, let's apply them to some model data to see how they compare.

In [32]:
df = pd.DataFrame(np.logspace(0, 1.05, 201), columns=['x'])
breaks = [3.5]
alphas = [4.5, 4]

df['y'] = BrokenPowerLaw(df['x'], 1, breaks, alphas)
df['y_smooth'] = SmoothlyBrokenPowerLaw(df['x'], 1, breaks, alphas, 0.1)
df

,x,y,y_smooth
0,1.000000,1.000000,1.000000
1,1.012162,1.055905,1.055905
2,1.024472,1.114936,1.114936
3,1.036931,1.177267,1.177267
4,1.049542,1.243083,1.243082
...,...,...,...
196,10.690549,24436.219760,24436.202474
197,10.820567,25646.849434,25646.833358
198,10.952166,26917.456642,26917.441691
199,11.085365,28251.012817,28250.998912


In [33]:
model = df.hvplot(x='y', y='x', label='Model').opts(color='blue', line_dash='dashed')#, logx=True, logy=True)
model_smooth = df.hvplot(x='y_smooth', y='x', label='Smooth Model').opts(color='magenta', line_dash='dashed')

break_lines = None
for br in breaks:
    if break_lines is None:
        break_lines = hv.HLine(br, label='Break = '+"{0:.{1}e}".format(br, 0)).opts(line_dash='dashed')
    else:
        break_lines = break_lines * hv.VLine(br, label='Break = '+"{0:.{1}e}".format(br, 0)).opts(line_dash='dashed')

(model * model_smooth * break_lines).opts(legend_position='top_left')     

:Overlay
   .Curve.Model                   :Curve   [y]   (x)
   .Curve.Smooth_Model            :Curve   [y_smooth]   (x)
   .HLine.Break_equals_4e_plus_00 :HLine   [x,y]